## Quickstart 

In this notebook you will learn how to set up inait's prediction tool and its basic functionalities.

We will use the classic Airline passenger dataset as a companion dataset but feel free to use your own data!

### Prerequisites

Let's import the relevant packages first.

In [ ]:
import sys
import pathlib

import pandas as pd

sys.path.append(pathlib.Path().resolve().parent.as_posix())

from inait import predict, plot, load_credentials

base_url, auth_key = load_credentials("../credentials.txt")

### Run your first prediction model

We will present an example using the classic Box & Jenkins dataset that contains the monthly number of airline passengers between 1949 and 1960 (source: Box, George EP, et al. Time series analysis: forecasting and control. John Wiley & Sons, 2015.). The goal is to predict the passengers for the next year, i.e., 12 steps ahead. 

In [ ]:
### Load and plot the data
data_path = "../data/airline.csv"
data = pd.read_csv(
    data_path, index_col=0
)  # dataset must have a valid datetime index with fixed frequency

plot(historical_data=data)

To forecast using inait models you need to specify the dataset (we expect a pandas DataFrame), how many steps in the future you want to forecast (the so-called `forecasting_horizon`), and how many steps in the past you want to use to make a prediction (`observation_length`).

In [ ]:
# Configure prediction parameters
target_columns = [
    "number_of_passengers"
]  # List of target columns to predict in the dataset
forecasting_horizon = 12  # Predict 12 time steps ahead
observation_length = 24  # Use last 24 time steps as historical context

In [ ]:
results = predict(
    base_url=base_url,
    auth_key=auth_key,
    data=data,
    target_columns=target_columns,
    forecasting_horizon=forecasting_horizon,
    observation_length=observation_length,
)

results["prediction"]

In [ ]:
plot(
    historical_data=data,
    predicted_data=results["prediction"],
    observation_length=len(data),
)

### Comments on results

The predictions appear to be well aligned with the historical series. However, since they extend into the future relative to our dataset, we cannot compare them directly to a ground truth. In the next notebook, we will explore how to split the dataset into training and testing sets in order to run a backtest and evaluate  how well the model would have performed in the past.

### Now it's your turn:
Experiment with the notebook by adjusting different parameters; for example, try changing `forecasting_horizon` or `observation_length` to see how the predictions behave.
You can also load your own dataset by uploading a file into the `data` folder and re-running the cells.